In [36]:
# Import required libraries and dependencies
import pandas as pd
from pathlib import Path
import hvplot.pandas
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import warnings 
warnings.filterwarnings("ignore")

In [37]:
# Load the data into a Pandas DataFrame
df_market_data = pd.read_csv(
   Path("Resources/crypto_market_data.csv"))

# Display sample data
df_market_data.head()

,coin_id,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
0,bitcoin,1.08388,7.60278,6.57509,7.67258,-3.25185,83.51840,37.51761
1,ethereum,0.22392,10.38134,4.80849,0.13169,-12.88890,186.77418,101.96023
2,tether,-0.21173,0.04935,0.00640,-0.04237,0.28037,-0.00542,0.01954
3,ripple,-0.37819,-0.60926,2.24984,0.23455,-17.55245,39.53888,-16.60193
4,bitcoin-cash,2.90585,17.09717,14.75334,15.74903,-13.71793,21.66042,14.49384


In [38]:
# Generate summary statistics
df_market_data.describe()

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
count,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000
mean,-0.269686,4.497147,0.185787,1.545693,-0.094119,236.537432,347.667956
std,2.694793,6.375218,8.376939,26.344218,47.365803,435.225304,1247.842884
min,-13.527860,-6.094560,-18.158900,-34.705480,-44.822480,-0.392100,-17.567530
25%,-0.608970,0.047260,-5.026620,-10.438470,-25.907990,21.660420,0.406170
50%,-0.063410,3.296410,0.109740,-0.042370,-7.544550,83.905200,69.691950
75%,0.612090,7.602780,5.510740,4.578130,0.657260,216.177610,168.372510
max,4.840330,20.694590,24.239190,140.795700,223.064370,2227.927820,7852.089700


In [39]:
# Plot your data to see what's in your DataFrame
df_market_data.hvplot.line(
    width=800,
    height=400,
    rot=90
)

:NdOverlay   [Variable]
   :Curve   [index]   (value)

---

### Prepare the Data

In [40]:
# Use the `StandardScaler()` module from scikit-learn to normalize the data from the CSV file
df_market_data_scaled = StandardScaler().fit_transform(df_market_data[['price_change_percentage_24h', 'price_change_percentage_7d', 'price_change_percentage_14d', 'price_change_percentage_30d', 'price_change_percentage_60d']])
df_market_data_scaled[0:5]

array([[ 0.50852937,  0.49319307,  0.77220043,  0.23545963, -0.0674951 ],
       [ 0.18544589,  0.93444504,  0.55869212, -0.05434093, -0.27348273],
       [ 0.02177396, -0.70633685, -0.02168042, -0.06103015,  0.00800452],
       [-0.04076438, -0.81092807,  0.24945797, -0.05038797, -0.37316402],
       [ 1.19303608,  2.00095907,  1.76061001,  0.54584206, -0.29120287]])

In [41]:
# Create a DataFrame with the scaled data
df_market_data_scaled = pd.DataFrame(
    df_market_data_scaled,
    columns=['price_change_percentage_24h', 'price_change_percentage_7d', 'price_change_percentage_14d', 'price_change_percentage_30d', 'price_change_percentage_60d'])

df_market_data_scaled['coin_id'] = df_market_data.coin_id

df_market_data_scaled = df_market_data_scaled.set_index('coin_id')

df_market_data_scaled.head()


,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d
coin_id,,,,,
bitcoin,0.508529,0.493193,0.772200,0.235460,-0.067495
ethereum,0.185446,0.934445,0.558692,-0.054341,-0.273483
tether,0.021774,-0.706337,-0.021680,-0.061030,0.008005
ripple,-0.040764,-0.810928,0.249458,-0.050388,-0.373164
bitcoin-cash,1.193036,2.000959,1.760610,0.545842,-0.291203


---

### Find the Best Value for k Using the Original Data.

In [73]:
# Create a list with the number of k-values from 1 to 11
# Create an empty list to store the inertia values
inertia = []
k = list(range(1, 11))


In [74]:
# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
# 2. Fit the model to the data using `df_market_data_scaled`
# 3. Append the model.inertia_ to the inertia list
for i in k:
    k_model = KMeans(n_clusters=i, random_state=1)
    k_model.fit(df_market_data_scaled)
    inertia.append(k_model.inertia_)

# Create a dictionary with the data to plot the Elbow curve
elbow_data = {'k': k, 'inertia': inertia}

# Create a DataFrame with the data to plot the Elbow curve
df_elbow = pd.DataFrame(elbow_data)

# Plot a line chart with all the inertia values computed with 
# the different values of k to visually identify the optimal value for k.
elbow_plot = df_elbow.hvplot.line(
    x='k',
    y='inertia',
    title='Elbow Curve',
    xticks=k
)
elbow_plot

:Curve   [k]   (inertia)

#### Answer the following question: 

**Question:** What is the best value for `k`?

**Answer:** '3 and 4'

---

### Cluster Cryptocurrencies with K-means Using the Original Data

In [45]:
# Initialize the K-Means model using the best value for k
# Fit the K-Means model using the scaled data
# Predict the clusters to group the cryptocurrencies using the scaled data
# Print the resulting array of cluster values.
model = KMeans(n_clusters=3)

model_clusters = model.fit_predict(df_market_data_scaled)

print(model_clusters)

[2 2 0 0 2 2 2 2 2 0 0 0 0 2 0 2 0 0 2 0 0 2 0 0 0 0 0 0 2 0 0 0 0 2 0 0 1
 0 0 0 0]


In [46]:
# Create a copy of the DataFrame
# Add a new column to the DataFrame with the predicted clusters
# Display sample data
df_market_data_scaled_predictions = df_market_data_scaled.copy()

df_market_data_scaled_predictions['predicted_clusters'] = model_clusters
df_market_data_scaled_predictions.head()


,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,predicted_clusters
coin_id,,,,,,
bitcoin,0.508529,0.493193,0.772200,0.235460,-0.067495,2
ethereum,0.185446,0.934445,0.558692,-0.054341,-0.273483,2
tether,0.021774,-0.706337,-0.021680,-0.061030,0.008005,0
ripple,-0.040764,-0.810928,0.249458,-0.050388,-0.373164,0
bitcoin-cash,1.193036,2.000959,1.760610,0.545842,-0.291203,2


In [77]:
# Create a scatter plot using hvPlot by setting 
# `x="price_change_percentage_24h"` and `y="price_change_percentage_7d"`. 
# Color the graph points with the labels found using K-Means and 
# add the crypto name in the `hover_cols` parameter to identify 
# the cryptocurrency represented by each data point.
scaled_predictions_plot = df_market_data_scaled_predictions.hvplot.scatter(
    x='price_change_percentage_24h',
    y='price_change_percentage_7d',
    by='predicted_clusters',
    hover_cols=['coin_id']
)
scaled_predictions_plot

:NdOverlay   [predicted_clusters]
   :Scatter   [price_change_percentage_24h]   (price_change_percentage_7d,coin_id)

---

### Optimize Clusters with Principal Component Analysis.

In [48]:
# Create a PCA model instance and set `n_components=3`.
# Use the PCA model with `fit_transform` to reduce to 
# three principal components.
# View the first five rows of the DataFrame. 
pca = PCA(n_components=3)
price_pca = pca.fit_transform(df_market_data_scaled_predictions)
price_pca[:5]

array([[ 1.38904717, -0.76487186, -0.15373556],
       [ 1.14571663, -1.25880294,  0.09820872],
       [-0.56335244,  0.63813954, -0.17074551],
       [-0.58267484,  0.46351646, -0.14824354],
       [ 2.75270269, -1.61045317, -0.54946512]])

In [61]:
# Retrieve the explained variance to determine how much information 
# can be attributed to each principal component.
import numpy as np
total_explained_var = np.sum(pca.explained_variance_ratio_)
total_explained_var

0.8969317707640904

#### Answer the following question: 

**Question:** What is the total explained variance of the three principal components?

**Answer:** Approx. 90% of the total variance is condensed into the 3 PCA variables.

In [50]:
# Create a new DataFrame with the PCA data.
# Creating a DataFrame with the PCA data
# Copy the crypto names from the original data
# Set the coinid column as index
# Display sample data
price_pca_df = pd.DataFrame(
    price_pca,
    columns=['PCA1', 'PCA2', 'PCA3'])

price_pca_df['coin_id'] = df_market_data.coin_id

price_pca_df = price_pca_df.set_index('coin_id')

price_pca_df.head()

,PCA1,PCA2,PCA3
coin_id,,,
bitcoin,1.389047,-0.764872,-0.153736
ethereum,1.145717,-1.258803,0.098209
tether,-0.563352,0.638140,-0.170746
ripple,-0.582675,0.463516,-0.148244
bitcoin-cash,2.752703,-1.610453,-0.549465


---

### Find the Best Value for k Using the PCA Data

In [51]:
# Create a list with the number of k-values from 1 to 11
# Create an empty list to store the inertia values
# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
# 2. Fit the model to the data using `df_market_data_pca`
# 3. Append the model.inertia_ to the inertia list

inertia = []
k = list(range(1, 11))

for i in k:
    k_model = KMeans(n_clusters=i, random_state=0)
    k_model.fit(price_pca_df)
    inertia.append(k_model.inertia_)

In [64]:
# Create a dictionary with the data to plot the Elbow curve
# Create a DataFrame with the data to plot the Elbow curve

pca_elbow_data = {'k': k, 'inertia': inertia}
pca_df_elbow = pd.DataFrame(elbow_data)
pca_df_elbow.head()


,k,inertia
0,1,215.460512
1,2,128.628147
2,3,72.534256
3,4,47.090607
4,5,36.615886


In [71]:
# Plot a line chart with all the inertia values computed with 
# the different values of k to visually identify the optimal value for k.
pca_elbow_plot = pca_df_elbow.hvplot.line(
    x='k',
    y='inertia',
    title='Elbow Curve',
    xticks=k
)
pca_elbow_plot

:Curve   [k]   (inertia)

#### Answer the following questions: 

* **Question:** What is the best value for `k` when using the PCA data?

  * **Answer:** 3 and 4


* **Question:** Does it differ from the best k value found using the original data?

  * **Answer:** Similar output

### Cluster Cryptocurrencies with K-means Using the PCA Data

In [66]:
# Initialize the K-Means model using the best value for k
# Fit the K-Means model using the PCA data
# Predict the clusters to group the cryptocurrencies using the PCA data
# Print the resulting array of cluster values.

model = KMeans(n_clusters=3, random_state=0)
model_clusters_2 = model.fit_predict(price_pca_df)
model_clusters_2

array([0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0,
       1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 2, 1, 1, 1, 1],
      dtype=int32)

In [67]:
# Create a copy of the DataFrame with the PCA data
# Add a new column to the DataFrame with the predicted clusters
# Display sample data

price_pca_df_predictions = price_pca_df.copy()
price_pca_df_predictions['predicted_clusters'] = k_3
price_pca_df_predictions.head()

,PCA1,PCA2,PCA3,predicted_clusters
coin_id,,,,
bitcoin,1.389047,-0.764872,-0.153736,1
ethereum,1.145717,-1.258803,0.098209,1
tether,-0.563352,0.638140,-0.170746,0
ripple,-0.582675,0.463516,-0.148244,0
bitcoin-cash,2.752703,-1.610453,-0.549465,1


In [76]:
# Create a scatter plot using hvPlot by setting 
# `x="PC1"` and `y="PC2"`. 
# Color the graph points with the labels found using K-Means and 
# add the crypto name in the `hover_cols` parameter to identify 
# the cryptocurrency represented by each data point.
price_predictions_plot = price_pca_df_predictions.hvplot.scatter(
    x='PCA1',
    y='PCA2',
    z='PCA3',
    by='predicted_clusters',
    hover_cols=['coin_id']
)
price_predictions_plot

:NdOverlay   [predicted_clusters]
   :Scatter   [PCA1]   (PCA2,coin_id)

In [69]:
# alernative plot option after research
import plotly.express as px

price_pca_df_predictions = price_pca_df_predictions.reset_index()
fig = px.scatter_3d(price_pca_df_predictions, x='PCA1', y='PCA2', z='PCA3',
                    color='predicted_clusters', hover_data=['coin_id'])

fig.show()

### Visualize and Compare the Results

In this section, you will visually analyze the cluster analysis results by contrasting the outcome with and without using the optimization techniques.

In [75]:
# Composite plot to contrast the Elbow curves
elbow_plot + pca_elbow_plot

:Layout
   .Curve.I  :Curve   [k]   (inertia)
   .Curve.II :Curve   [k]   (inertia)

In [78]:
# Composite plot to contrast the clusters
# YOUR CODE HERE!
scaled_predictions_plot + price_predictions_plot

:Layout
   .NdOverlay.I  :NdOverlay   [predicted_clusters]
      :Scatter   [price_change_percentage_24h]   (price_change_percentage_7d,coin_id)
   .NdOverlay.II :NdOverlay   [predicted_clusters]
      :Scatter   [PCA1]   (PCA2,coin_id)

#### Answer the following question: 

  * **Question:** After visually analyzing the cluster analysis results, what is the impact of using fewer features to cluster the data using K-Means?

  * **Answer:** After examining the cluster analysis results visually, there appears to be a significant impact to the clusters. Both elow plots are relatively similar. By implementing PPCA and using the optimal value for K of 3, the resulting plot was more accurate and precise in clustering the data.